# Cartographie des Musées de France : Une Analyse de la Répartition Géographique


La présente analyse se base sur le jeu de données "Liste et Localisation des Musées de France" disponible sur le site data.culture.gouv.fr à l'adresse https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin. Ce jeu de données fournit des informations détaillées sur les musées de France, notamment leur localisation, leur nom officiel, leur adresse, ainsi que leurs coordonnées géographiques.

L'objectif de cette analyse est de regrouper les musées de France selon un buffer carré de 1 km, de compter le nombre de musées dans chaque carré, puis d'agréger ces carrés par enveloppe de polygones s'ils se croisent. Cette approche permet d'obtenir une vision plus précise de la répartition géographique des musées en France.


## Méthodologie

La méthodologie utilisée dans cette analyse consiste en plusieurs étapes. Tout d'abord, nous créons un buffer carré de 1 km autour de chaque musée en utilisant les coordonnées géographiques fournies dans le jeu de données. Ensuite, nous comptons le nombre de musées dans chaque buffer. Enfin, nous agrégeons les buffers qui se croisent pour former des polygones représentant les zones de forte concentration de musées. Cette approche est mise en œuvre à l'aide d'une requête SQL utilisant les fonctionnalités spatiales de DuckDB.

Les résultats de cette analyse sont ensuite visualisés à l'aide d'une carte interactive créée avec Plotly Express, permettant de visualiser la répartition des musées en France de manière claire et intuitive. La carte représente les polygones obtenus après agrégation des buffers, avec une échelle de couleur indiquant le nombre de musées dans chaque zone.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
-- Création d'un buffer carré de 1km autour de chaque musée
musee_buffer AS (
  SELECT 
    "identifiant_museofile",
    ST_GeomFromText('POLYGON((' || 
      ST_X("geolocalisation") - 0.01 || ' ' || ST_Y("geolocalisation") - 0.01 || ', ' ||
      ST_X("geolocalisation") + 0.01 || ' ' || ST_Y("geolocalisation") - 0.01 || ', ' ||
      ST_X("geolocalisation") + 0.01 || ' ' || ST_Y("geolocalisation") + 0.01 || ', ' ||
      ST_X("geolocalisation") - 0.01 || ' ' || ST_Y("geolocalisation") + 0.01 || ', ' ||
      ST_X("geolocalisation") - 0.01 || ' ' || ST_Y("geolocalisation") - 0.01 || '))') AS buffer_geom
  FROM loaded_dataset
),
-- Comptage du nombre de musées dans chaque buffer
buffer_count AS (
  SELECT 
    buffer_geom,
    COUNT("identifiant_museofile") AS nb_musee
  FROM musee_buffer
  GROUP BY buffer_geom
),
-- Agrégation des buffers qui se croisent
buffer_union AS (
  SELECT 
    ST_Union_Agg(buffer_geom) AS polygon,
    SUM(nb_musee) AS nb_musee
  FROM buffer_count bc1
  WHERE EXISTS (
    SELECT 1 
    FROM buffer_count bc2 
    WHERE bc1.buffer_geom != bc2.buffer_geom AND ST_Intersects(bc1.buffer_geom, bc2.buffer_geom)
  )
  GROUP BY ST_AsText(bc1.buffer_geom)
)
-- Sélection des résultats
SELECT 
  ST_AsText(polygon) AS polygon_wkt,
  nb_musee
FROM buffer_union
ORDER BY nb_musee DESC """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 487 lignes


,polygon_wkt,nb_musee
0,"POLYGON ((4.808042 45.72319, 4.828042 45.72319...",3.0
1,"POLYGON ((7.741984 48.571229, 7.761984 48.5712...",3.0
2,"POLYGON ((3.205208 43.328908000000006, 3.22520...",2.0
3,"POLYGON ((4.061824 48.284239, 4.081824 48.2842...",2.0
4,"POLYGON ((1.088932 49.437268, 1.108932 49.4372...",2.0


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est une interface de haut niveau pour la visualisation de données avec Plotly. Cette technologie est adaptée car elle permet de créer des cartes interactives et personnalisables, idéales pour représenter des données géospatiales comme la répartition des musées sur un territoire. Le choix d'une carte choroplèthe permet de visualiser efficacement la densité de musées dans différentes zones.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import geopandas as gpd
import plotly.express as px
import shapely.wkt

gdf = gpd.GeoDataFrame(
    df,
    geometry=df["polygon_wkt"].apply(shapely.wkt.loads),
    crs=4326
)

gdf["centroid_lon"] = gdf.geometry.centroid.x
gdf["centroid_lat"] = gdf.geometry.centroid.y

colorscale = [(0, "#deebf7"), (1/3, "#9ecae1"), (2/3, "#4292c6"), (1, "#08519c")]

dataviz = px.choropleth_mapbox(
    gdf,
    geojson=gdf.geometry.__geo_interface__,
    locations=gdf.index,
    color="nb_musee",
    color_continuous_scale=colorscale,
    mapbox_style="carto-positron",
    zoom=4.8,
    center={"lat": 46.5, "lon": 2.5},
    opacity=0.65,
    hover_data={"nb_musee": True, "centroid_lon": False, "centroid_lat": False},
    labels={"nb_musee": "Nombre de musées"},
    title="Cartographie des musées - Répartition par buffer carré de 1 km"
)
dataviz.update_layout(margin=dict(l=20, r=20, t=40, b=20))
dataviz

C:\Users\ami\AppData\Local\Temp\ipykernel_13112\465449159.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroid_lon"] = gdf.geometry.centroid.x
C:\Users\ami\AppData\Local\Temp\ipykernel_13112\465449159.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroid_lat"] = gdf.geometry.centroid.y
C:\Users\ami\AppData\Local\Temp\ipykernel_13112\465449159.py:19: DeprecationWarning: *choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  dataviz = px.choropleth_mapbox(


---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250805_063703.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250805_063703.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250805_063703.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250805_063703.png


--> Image Plotly sauvegardée avec succès.
